# Generating training data composites 

These composites are for use in collecting training data. Implementing geomedian composites


### Load packages

In [1]:
%matplotlib inline

import numpy as np
import geopandas as gpd
import datacube
from odc.algo import to_f32, from_float, xr_geomedian
from datacube.utils import geometry
from datacube.helpers import write_geotiff
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_dask import create_local_dask_cluster
from deafrica_spatialtools import xr_rasterize
from deafrica_classificationtools import HiddenPrints
from deafrica_plotting import map_shapefile

### Set up a dask cluster

In [2]:
create_local_dask_cluster(aws_unsigned=True)

Client Scheduler: tcp://127.0.0.1:38247 Dashboard: /user/chad/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Connect to the datacube

In [3]:
dc = datacube.Datacube(app='Geomedian_composites')

## Analysis Parameters

In [4]:
time_range = ('2019')

vector_file = 'data/tiles.shp'
attribute_col = 'id'
aez_region = 'data/AEZs/Western.shp'

products = ['s2a_msil2a', 's2b_msil2a']
measurements = ['red', 'green', 'blue', 'nir_1']
resolution = (-20, 20)
output_crs = 'EPSG:6933'
align = (0,0)
dask_chunks= {'x':500,'y':500, 'time':1}

export_prefix="data/training_validation/geomedians/Western/Western_S2_geomedian_tile-"

### Set up a query

In [5]:
# Create a reusable query
query = {'time': time_range,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         'dask_chunks':dask_chunks
         }

query

{'time': '2019',
 'measurements': ['red', 'green', 'blue', 'nir_1'],
 'resolution': (-20, 20),
 'output_crs': 'EPSG:6933',
 'align': (0, 0),
 'dask_chunks': {'x': 500, 'y': 500, 'time': 1}}

### Open and clip shapefiles

Open `tiles.shp` and clip it to the boundary shapefile (in this example the `aez_region`)


In [6]:
#read shapefile
gdf = gpd.read_file(vector_file)

#open shapefile
aez=gpd.read_file(aez_region)

# clip points to region
gdf = gpd.overlay(gdf, aez, how='intersection')

### Add a unique identifier

and plot the tiles using `map_shapefile`, you can hover your mouse over the tiles to see the ID 

In [7]:
# add an ID column
gdf[attribute_col]=range(0, len(gdf))

#print gdf
map_shapefile(gdf, attribute_col)

Label(value='')

Map(basemap={'url': 'http://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/…

#### Enter a list of tile indexes to load

In [8]:
tiles_to_load = [40,56,39,58]

### Generate a geomedian composite

Load data for each tile, calculate geomedian using `xr_geomedian` and export file as geotiff

In [ ]:
%%time

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[gdf['id'].loc[tiles_to_load]].iterrows():
    
    print("Working on tile: "+str(gdf['id'][index]))
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    with HiddenPrints():
        ds = load_ard(dc=dc, 
                          products=products,                 
                          group_by='solar_day',
                          **query)
    
    # Generate a polygon mask to keep only data within the polygon
#     with HiddenPrints():
#         mask = xr_rasterize(gdf.iloc[[index]], ds)
    
#     # Mask dataset to set pixels outside the polygon to `NaN`
#     ds = ds.where(mask)
    
    sr_max_value = 65455                 # maximum value for SR in the loaded product
    scale, offset = (1/sr_max_value, 0)  # differs per product, aim for 0-1 values in float32

    #scale the values using the f_32 util function
    ds_scaled = to_f32(ds,
                       scale=scale,
                       offset=offset)
    #generate a geomedian
    geomedian = xr_geomedian(ds_scaled, 
                             num_threads=1,  
                             eps=1e-7,  
                             nocheck=True)
    
    geomedian = geomedian.compute()
      
    #convert SR scaling values back to original values
    geomedian = from_float(geomedian, 
                           dtype='float32', 
                           nodata=np.nan, 
                           scale=1/scale, 
                           offset=-offset/scale)
        
    print("     Writing to file...")
    write_geotiff(export_prefix+str(row[attribute_col]) + '.tif', 
                  geomedian)
    

Working on tile: 40
     Writing to file...
Working on tile: 56


In [1]:
import xarray as xr

In [2]:
xr.open_rasterio("data/training_validation/geomedians/Western/Western_S2_geomedian_tile-40.tif")

<xarray.DataArray (band: 4, y: 7500, x: 7500)>
[225000000 values with dtype=float32]
Coordinates:
  * band     (band) int64 1 2 3 4
  * y        (y) float64 9e+05 9e+05 9e+05 ... 7.500e+05 7.5e+05 7.5e+05
  * x        (x) float64 -3e+05 -3e+05 -3e+05 ... -1.500e+05 -1.5e+05 -1.5e+05
Attributes:
    transform:      (20.0, 0.0, -300000.0, 0.0, -20.0, 900000.0)
    crs:            +init=epsg:6933
    res:            (20.0, 20.0)
    is_tiled:       1
    nodatavals:     (0.0, 0.0, 0.0, 0.0)
    scales:         (1.0, 1.0, 1.0, 1.0)
    offsets:        (0.0, 0.0, 0.0, 0.0)
    AREA_OR_POINT:  Area

---
Code to get geojson from datacube UI 

In [ ]:
# from datacube import Datacube
# from odc.ui import DcViewer
# dc = Datacube()
# ui = DcViewer(dc, "2018", products=['ga_ls8c_gm_2_annual'])
# ui

# x = ui._gui.map.layers[1].data

# from shapely.geometry import shape
# geom = [shape(i) for i in x['features']]
# gdf = gpd.GeoDataFrame({'geometry':geom}, crs='EPSG:4326')
# gdf.to_file('data/tiles.shp')